Try to reload models -- Test only.


In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
import cv2
import numpy as np


In [4]:
import glob
vehicle_images=glob.glob("dataset/vehicles/*/*.png")
non_vehicle_images=glob.glob("dataset/non-vehicles/*/*.png")
print("Dataset Contains: {} Vehicle and {} Non-Vehicle images".format(len(vehicle_images),len(non_vehicle_images)))

from skimage.feature import hog
def get_features(img,cell_per_block=2,pix_per_cell=8,orient=9, colorspace='HSV', color_hist_bins=16, img_resize=(32,32)):
#     returns the feature vector for the image
    if(img_resize != (img.shape[0]),img.shape[1]):
        img=cv2.resize(img,img_resize)
#     get hog features before colorspace conversion
    feat=hog(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY), orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell), cells_per_block=(cell_per_block, cell_per_block), visualise=False, feature_vector=True)
#     Convert to required colorspace
    if(colorspace!='BGR'):
        if(colorspace=='HSV'):
            img=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
        elif(colorspace=='HLS'):
            img=cv2.cvtColor(img,cv2.COLOR_BGR2HLS)
        else:
            pass    
    for i in range(3):
#         loop over three channels
        ch_hist = np.histogram(img[:,:,i],bins=color_hist_bins,range=(0,256))[0]
        feat = np.concatenate((feat,ch_hist))
    return feat

vehicle_features=[]
for vehicle_image in vehicle_images:
    vehicle_features.append(get_features(cv2.imread(vehicle_image)))
non_vehicle_features=[]
for non_vehicle_image in non_vehicle_images:
    non_vehicle_features.append(get_features(cv2.imread(non_vehicle_image)))

from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

X_raw = np.vstack((vehicle_features,non_vehicle_features)).astype(np.float64)
y_raw = np.hstack((np.ones(len(vehicle_features)),np.zeros(len(non_vehicle_features))))

stdscaler = StandardScaler().fit(X_raw)
X_raw_scaled = stdscaler.transform(X_raw)

# Test train split
X_train, X_test, y_train, y_test = train_test_split(
    X_raw_scaled, y_raw, test_size=0.2, random_state=12)


    

Dataset Contains: 8792 Vehicle and 8968 Non-Vehicle images


/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
from sklearn.externals import joblib
clf=joblib.load('model.pkl')
clf.score(X_train,y_train)

0.99985923423423428

In [9]:
clf.score(X_test,y_test)

0.99099099099099097

In [14]:
joblib.dump(np.array([X_train,y_train,X_test,y_test]),'test.pkl')

['test.pkl']

In [15]:
ll=joblib.load('test.pkl')

In [17]:
m=ll[0]

In [18]:
m.shape

(14208, 372)

In [19]:
ll[1].shape

(14208,)

In [20]:
ll[2].shape

(3552, 372)

In [2]:
from sklearn.externals import joblib
l=joblib.load('dataset_bkp.pkl')
X_train=l[0]

In [3]:
X_train.shape


(14208, 372)